# ipyautoui

wrapper that sits on top of ipywidgets and other ipy widget libraries to template / automate the creation of widget forms. Uses pydantic to create defined data-container and serialisation to JSON. Includes example patterns for adding new custom widgets.



## Create a UI object

ipyautoui creates a row of widgets with a: `name`, `value` and `label`. The `value` is interpreted by the WidgetRowBase class and a widget is widget type is guess by the type of the value and the `kwargs` that can also be passed when initialising the widget. 

In [1]:
from ipyautoui.autoui import AutoUi, AutoUiBase, WidgetRowBase

?WidgetRowBase

Init signature:
WidgetRowBase(
    *,
    value: Any = None,
    kwargs: Dict = {},
    autoui_type: str = None,
    widget: Any = None,
    name: str = 'name',
    label: str = 'label',
) -> None
Docstring:     
data container for a AutoWidget. 
this is a data container for all auto-widgets.
it inherits pydantic.BaseModel and uses pydantic validation tools to 
initiate the "widget" and the "autoui_type" based on the value type and kwargs. 
using the pydantic ".dict()" and ".json()" commands data can be serialised. the "widget"
parameter is excluded from the "dict()" and "json()" commands; this means that the 
output contains strings only and can be written to file.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           /mnt/c/engDev/git_mf/ipyautoui/ipyautoui/autoui.py
Type:           ModelMetaclass
Subclasses:     WidgetRow


In [2]:
rows = [
    {'value': 1, 'name':'integer'},
    {'value': 'string'}, 
    {'value': 1, 'kwargs': {'min':0, 'max':4}}
]

aui = AutoUiBase(rows = rows)
ui = AutoUi(aui)
ui

In [3]:
# AutoUi class adds the widget attribute
# it uses traitlets to ensure that if the widget value
# changes the ui.value attribute also changes with it
display(ui.rows[0].widget)
print(ui.rows[0].widget.value)
ui.rows[0].widget.value = 2
ui.rows[0].value

IntText(value=1)

1


2

## Output / transport data from UI

In [4]:
## using Pydantic data serialisation amd config option introduced in v1.8.3,
# the JSON inconpatible attributes are removed
# when the widget is written to dict or JSON. 
ui.dict()

{'rows': [{'value': 2,
   'kwargs': {},
   'autoui_type': "<class 'ipywidgets.widgets.widget_int.IntText'>",
   'name': 'integer',
   'label': 'label'},
  {'value': 'string',
   'kwargs': {},
   'autoui_type': "<class 'ipywidgets.widgets.widget_string.Text'>",
   'name': 'name',
   'label': 'label'},
  {'value': 1,
   'kwargs': {'min': 0, 'max': 4},
   'autoui_type': "<class 'ipywidgets.widgets.widget_int.IntSlider'>",
   'name': 'name',
   'label': 'label'}]}

## Reload data back into UI

the `autoui_type` saves a class string defining what widget should be used. This is converted to the object using importlib.

In [5]:
#  importlib is used to create the widget from the `autoui_type` string characteristic
aui = AutoUiBase(**ui.dict())
AutoUi(aui)

## Create custom UI object rows

This library mainly relies on ipywidgets, but other libraries can also be used. The AutoUi widget uses traitlets to watch when the `widget.value` changes, wrappers can be made around other widget libraries to allow this behaviour. 

In this way pandas dataframes using ipydatagrid can also be used.

(See `ipyautoui.custom_widgets` for more examples.



In [7]:
import pandas as pd
rows.append(
    {'name':'pandas', 'value': pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]}), 'label': 'example of a custom widget'}
)

In [8]:

aui = AutoUiBase(rows = rows)
ui = AutoUi(aui)
ui

## Write to file